# Imports 

In [51]:
import pandas as pd
import datetime
import time
import warnings
import math
import statistics
import numpy as np
import scipy.stats
import openpyxl
warnings.filterwarnings('ignore')

In [2]:
scraping = pd.read_csv('Scraping_data.csv')
api = pd.read_csv('API_data.csv', names = ['Duration','Device','In','Mac','Out','Light Sleep','Deep Sleep','REM Sleep','Score','Snoring duration','Snoring episodes','BPM','Awake','Status','Regularity','Recovery','Interruptions','TTS'])
sensor = pd.read_csv('Sensor_data.csv')

# Cleaning

## Scraping

In [3]:
# inital dataframe
scraping.head()

,Unnamed: 0,Date,Pattern,Time
0,0,28 November 2021,Awake,01:13-01:42
1,1,28 November 2021,Deep,01:42-02:58
2,2,28 November 2021,Light,02:58-03:16
3,3,28 November 2021,Deep,03:16-03:30
4,4,28 November 2021,Light,03:30-04:04


In [4]:
# remove useless sscolumn
scraping = scraping.drop('Unnamed: 0', 1)

In [5]:
# split period start and end time into two columns
scraping[['Start', 'End']] = scraping['Time'].str.split('-', expand=True)

In [6]:
# drop the time column
scraping = scraping.drop('Time', 1)

In [7]:
# full date/time to use as timeseries after
scraping['Start'] = scraping['Date'] + scraping['Start']
scraping['End'] = scraping['Date'] + ' ' + scraping['End']

In [8]:
# drop useless column
scraping = scraping.drop('Date',1)

In [9]:
# formating the date and time as datetime for use as time series
scraping['Start'] = pd.to_datetime(scraping['Start'], format = '%d %B %Y %H:%M')
scraping['End'] = pd.to_datetime(scraping['End'], format = '%d %B %Y %H:%M')

In [10]:
# verifying type
type(scraping['Start'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
scraping.head()

,Pattern,Start,End
0,Awake,2021-11-28 01:13:00,2021-11-28 01:42:00
1,Deep,2021-11-28 01:42:00,2021-11-28 02:58:00
2,Light,2021-11-28 02:58:00,2021-11-28 03:16:00
3,Deep,2021-11-28 03:16:00,2021-11-28 03:30:00
4,Light,2021-11-28 03:30:00,2021-11-28 04:04:00


## API

In [12]:
# drop useless columns
api = api.drop(['Device','Mac','Snoring duration','Snoring episodes','Status','Regularity','Recovery'], 1)

In [13]:
api.head()

,Duration,In,Out,Light Sleep,Deep Sleep,REM Sleep,Score,BPM,Awake,Interruptions,TTS
0,SleepDuration: 07:16:00,"InBedDateandTime: November 11, 2021 at 12:30AM","OutofBedDateandTime: November 11, 2021 at 08:21AM",LightSleepDuration: 04:18:00,DeepSleepDuration: 02:14:00,RemSleepDuration: 00:44:00,SleepScore: 85,HeartRateAverage: 57,AwakeDuration: 00:28:00,NbInterruptions: 5,TimeToSleep: 00:11:00
1,SleepDuration: 06:16:00,"InBedDateandTime: November 12, 2021 at 12:54AM","OutofBedDateandTime: November 12, 2021 at 07:53AM",LightSleepDuration: 03:05:00,DeepSleepDuration: 02:03:00,RemSleepDuration: 01:08:00,SleepScore: 72,HeartRateAverage: 56,AwakeDuration: 00:43:00,NbInterruptions: 1,TimeToSleep: 00:13:00
2,SleepDuration: 07:11:00,"InBedDateandTime: November 13, 2021 at 12:10AM","OutofBedDateandTime: November 13, 2021 at 08:25AM",LightSleepDuration: 03:31:00,DeepSleepDuration: 02:35:00,RemSleepDuration: 01:05:00,SleepScore: 86,HeartRateAverage: 59,AwakeDuration: 00:57:00,NbInterruptions: 4,TimeToSleep: 00:41:00
3,SleepDuration: 06:39:00,"InBedDateandTime: November 14, 2021 at 12:14AM","OutofBedDateandTime: November 14, 2021 at 07:50AM",LightSleepDuration: 03:18:00,DeepSleepDuration: 02:03:00,RemSleepDuration: 01:18:00,SleepScore: 80,HeartRateAverage: 53,AwakeDuration: 00:55:00,NbInterruptions: 2,TimeToSleep: 00:48:00
4,SleepDuration: 06:52:00,"InBedDateandTime: November 14, 2021 at 11:38PM","OutofBedDateandTime: November 15, 2021 at 08:04AM",LightSleepDuration: 03:44:00,DeepSleepDuration: 02:00:00,RemSleepDuration: 01:08:00,SleepScore: 77,HeartRateAverage: 58,AwakeDuration: 01:34:00,NbInterruptions: 3,TimeToSleep: 00:52:00


In [14]:
# remove unwanted text
for i in range(len(api)):
    api['Duration'][i] = api['Duration'][i].split('SleepDuration:')[-1]
    api['In'][i] = api['In'][i].split('InBedDateandTime:')[-1]
    api['Out'][i] = api['Out'][i].split('OutofBedDateandTime:')[-1]
    api['Light Sleep'][i] = api['Light Sleep'][i].split('LightSleepDuration:')[-1]
    api['Deep Sleep'][i] = api['Deep Sleep'][i].split('DeepSleepDuration:')[-1]
    api['REM Sleep'][i] = api['REM Sleep'][i].split('RemSleepDuration:')[-1]
    api['Score'][i] = api['Score'][i].split('SleepScore:')[-1]
    api['BPM'][i] = api['BPM'][i].split('HeartRateAverage:')[-1]
    api['Awake'][i] = api['Awake'][i].split('AwakeDuration:')[-1]
    api['Interruptions'][i] = api['Interruptions'][i].split('NbInterruptions:')[-1]
    api['TTS'][i] = api['TTS'][i].split('TimeToSleep:')[-1]

In [15]:
# remove leading or trailing whitespaces
for col in api.columns:
    api[col] = api[col].str.strip()

In [16]:
# format datetime
api['In'] = pd.to_datetime(api['In'], format = '%B %d, %Y at %I:%M%p')
api['Out'] = pd.to_datetime(api['Out'], format = '%B %d, %Y at %I:%M%p')

In [17]:
#make durations as integers (minutes)
ts = ['Duration','Light Sleep', 'Deep Sleep', 'REM Sleep', 'Awake', 'TTS']

for col in ts:
    for i in range(len(api)):
        hour = int(api[col][i].split(':')[0])
        minute = int(api[col][i].split(':')[1])
        api[col][i] = hour*60 + minute

In [18]:
# make sur int columns are good type
api[["Score", "BPM", "Interruptions"]] = api[["Score", "BPM", "Interruptions"]].apply(pd.to_numeric)

In [19]:
api['Date'] = pd.to_datetime(api['Out']).dt.to_period('D')

In [20]:
api.head()

,Duration,In,Out,Light Sleep,Deep Sleep,REM Sleep,Score,BPM,Awake,Interruptions,TTS,Date
0,436,2021-11-11 00:30:00,2021-11-11 08:21:00,258,134,44,85,57,28,5,11,2021-11-11
1,376,2021-11-12 00:54:00,2021-11-12 07:53:00,185,123,68,72,56,43,1,13,2021-11-12
2,431,2021-11-13 00:10:00,2021-11-13 08:25:00,211,155,65,86,59,57,4,41,2021-11-13
3,399,2021-11-14 00:14:00,2021-11-14 07:50:00,198,123,78,80,53,55,2,48,2021-11-14
4,412,2021-11-14 23:38:00,2021-11-15 08:04:00,224,120,68,77,58,94,3,52,2021-11-15


## Sensor

In [21]:
sensor.head()

,_id,sensors.humidity,sensors.light,sensors.temperature,ts
0,619007b5a4c07eea3a262a7b,62.2,1256,20.0,2021-11-13T18:45:09.004Z
1,61900cc81d8f09b84c530ac7,62.6,1755,20.0,2021-11-13T19:06:48.147Z
2,61900f21118a094f4cc36b14,64.7,1747,19.8,2021-11-13T19:16:49.476Z
3,6190117a118a094f4cc54655,65.4,1736,19.5,2021-11-13T19:26:50.661Z
4,619013d3a4c07eea3a2f9bee,65.1,1738,19.4,2021-11-13T19:36:51.835Z


In [22]:
# drop useless column
sensor = sensor.drop('_id', 1)

In [23]:
# rename other columns
sensor = sensor.rename({'sensors.humidity': 'Humidity', 'sensors.light': 'Light', 'sensors.temperature' : 'Temperature', 'ts' : 'Time stamp'}, axis=1)

In [24]:
# create date and time variables
date = sensor['Time stamp'].str.split('T', expand=True)[0]
time = sensor['Time stamp'].str.split('T', expand=True)[1].str.split('.', expand=True)[0]

In [25]:
# modify time stamp column
sensor['Time stamp'] = date + ' ' + time

In [26]:
# format to datetime
sensor['Time stamp'] = pd.to_datetime(sensor['Time stamp'], format = '%Y-%m-%d %H:%M:%S')

In [27]:
sensor.head()

,Humidity,Light,Temperature,Time stamp
0,62.2,1256,20.0,2021-11-13 18:45:09
1,62.6,1755,20.0,2021-11-13 19:06:48
2,64.7,1747,19.8,2021-11-13 19:16:49
3,65.4,1736,19.5,2021-11-13 19:26:50
4,65.1,1738,19.4,2021-11-13 19:36:51


## All datasets 

In [28]:
scraping.head()

,Pattern,Start,End
0,Awake,2021-11-28 01:13:00,2021-11-28 01:42:00
1,Deep,2021-11-28 01:42:00,2021-11-28 02:58:00
2,Light,2021-11-28 02:58:00,2021-11-28 03:16:00
3,Deep,2021-11-28 03:16:00,2021-11-28 03:30:00
4,Light,2021-11-28 03:30:00,2021-11-28 04:04:00


In [29]:
api.head()

,Duration,In,Out,Light Sleep,Deep Sleep,REM Sleep,Score,BPM,Awake,Interruptions,TTS,Date
0,436,2021-11-11 00:30:00,2021-11-11 08:21:00,258,134,44,85,57,28,5,11,2021-11-11
1,376,2021-11-12 00:54:00,2021-11-12 07:53:00,185,123,68,72,56,43,1,13,2021-11-12
2,431,2021-11-13 00:10:00,2021-11-13 08:25:00,211,155,65,86,59,57,4,41,2021-11-13
3,399,2021-11-14 00:14:00,2021-11-14 07:50:00,198,123,78,80,53,55,2,48,2021-11-14
4,412,2021-11-14 23:38:00,2021-11-15 08:04:00,224,120,68,77,58,94,3,52,2021-11-15


In [30]:
sensor.head()

,Humidity,Light,Temperature,Time stamp
0,62.2,1256,20.0,2021-11-13 18:45:09
1,62.6,1755,20.0,2021-11-13 19:06:48
2,64.7,1747,19.8,2021-11-13 19:16:49
3,65.4,1736,19.5,2021-11-13 19:26:50
4,65.1,1738,19.4,2021-11-13 19:36:51


# Analysis

1. need to delete sensor data not in the sleeping time frame
2. need to average out light, humidity and temperature
3. time series visualisation
4. simple correlations
5. create model to determine best sleeping criteria based on deep sleep and also score (one specific and one average out)


## Restrict sensor data

In [31]:
sensor_new = pd.DataFrame(columns = ['Humidity','Light','Temperature','Time stamp'])
humidity = pd.DataFrame(columns = ['Out','Mean', 'Median', 'Std', 'Variance', 'Min', 'Max', 'Range'])
temperature = pd.DataFrame(columns = ['Out','Mean', 'Median', 'Std', 'Variance', 'Min', 'Max', 'Range'])
light = pd.DataFrame(columns = ['Out','Mean', 'Median', 'Std', 'Variance', 'Min', 'Max', 'Range'])

In [32]:
for i in range(len(api)):
    for j in range(len(sensor)):
        if api['In'][i] <= sensor['Time stamp'][j] <= api['Out'][i]:
            sensor_new = sensor_new.append(sensor.loc[[j]])
        else:
            pass

In [33]:
# well the light goes directly to 0 at night ... logical but i guess that's what happens
sensor_new = sensor_new.reset_index()
sensor_new = sensor_new.drop('index',1)
sensor_new.head()

,Humidity,Light,Temperature,Time stamp
0,63.6,310,19.4,2021-11-14 00:17:29
1,63.0,0,19.4,2021-11-14 00:27:31
2,64.0,0,19.4,2021-11-14 00:37:32
3,64.5,0,19.2,2021-11-14 00:47:34
4,64.4,0,19.3,2021-11-14 00:57:35


In [34]:
hum_lst = []
temp_lst = []
light_lst = []

In [35]:
for i in range(len(api)):
    hum_lst = []
    temp_lst = []
    light_lst = []
    for j in range(len(sensor_new)):
        if api['In'][i] <= sensor_new['Time stamp'][j] <= api['Out'][i]:
            hum_lst.append(sensor_new['Humidity'][j])
            temp_lst.append(sensor_new['Temperature'][j])
            light_lst.append(sensor_new['Light'][j])
        else:
            pass
    if len(hum_lst) != 0:
            
        hum_metrics = [api['Out'][i],statistics.mean(hum_lst),statistics.median(hum_lst),statistics.stdev(hum_lst),
                       statistics.variance(hum_lst),min(hum_lst),max(hum_lst),max(hum_lst)-min(hum_lst)]
        temp_metrics = [api['Out'][i],statistics.mean(temp_lst),statistics.median(temp_lst),statistics.stdev(temp_lst),
                       statistics.variance(temp_lst),min(temp_lst),max(temp_lst),max(temp_lst)-min(temp_lst)]
        light_metrics = [api['Out'][i],statistics.mean(light_lst),statistics.median(light_lst),statistics.stdev(light_lst),
                       statistics.variance(light_lst),min(light_lst),max(light_lst),max(light_lst)-min(light_lst)]
    
        hum_length = len(humidity)
        temp_length = len(temperature)
        light_length = len(light)
        
        humidity.loc[hum_length] = hum_metrics
        temperature.loc[temp_length] = temp_metrics
        light.loc[light_length] = light_metrics


In [36]:
humidity['Date'] = pd.to_datetime(humidity['Out']).dt.to_period('D')
temperature['Date'] = pd.to_datetime(temperature['Out']).dt.to_period('D')
light['Date'] = pd.to_datetime(light['Out']).dt.to_period('D')

In [37]:
humidity = humidity.drop('Out',1)
temperature = temperature.drop('Out',1)
light = light.drop('Out',1)

In [38]:
humidity

,Mean,Median,Std,Variance,Min,Max,Range,Date
0,66.602174,66.10,1.788667,3.199329,63.0,69.7,6.7,2021-11-14
1,74.137255,74.30,2.078072,4.318384,69.7,77.2,7.5,2021-11-15
2,67.547368,68.10,1.805657,3.260398,63.8,69.3,5.5,2021-11-16
3,67.032000,67.35,1.702272,2.897731,62.8,69.6,6.8,2021-11-17
4,69.626667,69.70,1.120884,1.256381,67.8,71.6,3.8,2021-11-18


In [39]:
temperature

,Mean,Median,Std,Variance,Min,Max,Range,Date
0,19.454348,19.1,0.816417,0.666536,18.9,21.3,2.4,2021-11-14
1,19.541176,19.2,0.694601,0.482471,19.0,21.1,2.1,2021-11-15
2,19.292105,18.7,0.977139,0.954801,18.6,21.2,2.6,2021-11-16
3,18.236000,17.9,1.063064,1.130106,16.8,21.6,4.8,2021-11-17
4,18.580000,18.5,0.197122,0.038857,18.3,19.0,0.7,2021-11-18


# Exports

In [47]:
sensor.to_csv('Final_data/sensor.csv',index=False)
sensor_new.to_csv('Final_data/sensor_lim.csv',index=False)
api.to_csv('Final_data/api.csv',index=False)
scraping.to_csv('Final_data/scraping.csv',index=False)
light.to_csv('Final_data/light.csv',index=False)
temperature.to_csv('Final_data/temperature.csv',index=False)
humidity.to_csv('Final_data/humidity.csv',index=False)

In [53]:
# sensor.to_excel('Excel/sensor.xlsx',index=False)
# sensor_new.to_excel('Excel/sensor_lim.xlsx',index=False)
# api.to_excel('Excel/api.xlsx',index=False)
# scraping.to_excel('Excel/scraping.xlsx',index=False)
# light.to_excel('Excel/light.xlsx',index=False)
# temperature.to_excel('Excel/temperature.xlsx',index=False)
# humidity.to_excel('Excel/humidity.xlsx',index=False)